# PDCLMBase Model Training Test

Bu notebook PDCLMBase modelinin pretraining işlemini test eder.

Hedefler:
1. Model import ve initialization
2. WikiText verisi ile 100 iterasyon training
3. Loss tracking ve visualization
4. Final loss < 1.0 hedefi

In [ ]:
# Import libraries
import sys
import os
sys.path.append('..')

import torch
import torch.nn as nn
from torch.optim import AdamW
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Import our model and utilities
from src.model import PDCLMBase, pretrain_step, create_batches
from src.utils import load_wikitext_data, visualize_training_curve

print("✅ Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Load WikiText data
data_path = "../data/raw/wikitext_sample.txt"
print(f"📖 Loading data from: {data_path}")

if os.path.exists(data_path):
    raw_text = load_wikitext_data(data_path)
    print(f"✅ Data loaded successfully")
    print(f"📏 Text length: {len(raw_text):,} characters")
    print(f"📝 Sample: {raw_text[:100]}...")
else:
    print(f"❌ Data file not found: {data_path}")
    # Create sample data for testing
    raw_text = "This is a sample text for testing the PDCLM model. " * 1000
    print(f"📝 Using synthetic data: {len(raw_text):,} characters")

In [ ]:
# Initialize model
print("🤖 Initializing PDCLMBase model...")

# Model configuration
embed_dim = 256
num_layers = 4
heads = 4
window_size = 512

print(f"Configuration:")
print(f"  - embed_dim: {embed_dim}")
print(f"  - num_layers: {num_layers}")
print(f"  - heads: {heads}")
print(f"  - window_size: {window_size}")

# Create model
model = PDCLMBase(
    embed_dim=embed_dim,
    num_layers=num_layers,
    heads=heads,
    window_size=window_size
)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"✅ Model created and moved to {device}")
print(f"📊 Model parameters: {model.count_parameters():,}")
print(f"📋 Model info: {model.get_model_info()}")

In [ ]:
# Initialize optimizer
learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate)

print(f"🔧 Optimizer configured:")
print(f"  - Learning rate: {learning_rate}")
print(f"  - Optimizer: AdamW")
print(f"  - Device: {device}")

In [ ]:
# Training parameters
batch_size = 10000  # characters per batch
num_iterations = 100
log_interval = 20

print(f"🎯 Training configuration:")
print(f"  - Batch size: {batch_size:,} characters")
print(f"  - Iterations: {num_iterations}")
print(f"  - Log interval: {log_interval}")

# Create batches from text
print(f"📦 Creating batches...")
batches = list(create_batches(raw_text, batch_size=batch_size))
print(f"✅ Created {len(batches)} batches")

In [ ]:
# Training loop
print("🚀 Starting pretraining...")
losses = []

model.train()
for iteration in range(num_iterations):
    # Select batch (cycle through available batches)
    batch_text = batches[iteration % len(batches)]
    
    # Training step
    try:
        loss = pretrain_step(model, batch_text, optimizer, device)
        losses.append(loss)
        
        # Logging
        if iteration % log_interval == 0:
            print(f"Iteration {iteration:3d}/{num_iterations} | Loss: {loss:.6f}")
            
        # Check for NaN
        if np.isnan(loss):
            print(f"❌ NaN loss detected at iteration {iteration}")
            break
            
    except Exception as e:
        print(f"❌ Error at iteration {iteration}: {str(e)}")
        break

print(f"\n✅ Training completed!")
print(f"📊 Total iterations: {len(losses)}")
print(f"📈 Final loss: {losses[-1]:.6f}")
print(f"📉 Best loss: {min(losses):.6f}")

In [ ]:
# Loss visualization
print("📊 Creating loss plot...")

plt.figure(figsize=(12, 6))
plt.plot(losses, 'b-', linewidth=2, label='Training Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('PDCLMBase Pretraining Loss Curve')
plt.grid(True, alpha=0.3)
plt.legend()

# Add loss statistics
final_loss = losses[-1]
min_loss = min(losses)
plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.7, label='Target Loss (1.0)')
plt.text(0.02, 0.98, f'Final: {final_loss:.4f}\nMin: {min_loss:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.legend()
plt.tight_layout()

# Save plot
plot_path = "pretrain_loss.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"💾 Loss plot saved: {plot_path}")

plt.show()

In [ ]:
# Model validation
print("🔍 Validating trained model...")

model.eval()
test_text = "This is a test string for validation. " * 50  # ~1500 chars

with torch.no_grad():
    val_loss = model(test_text)
    print(f"✅ Validation loss: {val_loss.item():.6f}")
    
    # Check if loss is in reasonable range
    is_reasonable = 0 < val_loss.item() < 100
    has_nan = torch.isnan(val_loss)
    
    print(f"📊 Validation results:")
    print(f"  - Loss value: {val_loss.item():.6f}")
    print(f"  - Has NaN: {has_nan.item()}")
    print(f"  - Is reasonable (0-100): {is_reasonable}")
    print(f"  - Target achieved (loss < 1.0): {val_loss.item() < 1.0}")

In [ ]:
# Training summary
print("\n" + "="*50)
print("🏁 TRAINING SUMMARY")
print("="*50)

final_loss = losses[-1] if losses else float('inf')
min_loss = min(losses) if losses else float('inf')
success = final_loss < 1.0 and not np.isnan(final_loss)

print(f"📊 Training Results:")
print(f"  - Completed iterations: {len(losses)}/{num_iterations}")
print(f"  - Final loss: {final_loss:.6f}")
print(f"  - Best loss: {min_loss:.6f}")
print(f"  - Loss reduction: {((losses[0] - final_loss) / losses[0] * 100):.1f}%")

print(f"\n🎯 Goals:")
print(f"  - Final loss < 1.0: {'✅' if final_loss < 1.0 else '❌'}")
print(f"  - No NaN values: {'✅' if not np.isnan(final_loss) else '❌'}")
print(f"  - Training completed: {'✅' if len(losses) == num_iterations else '❌'}")

print(f"\n🏆 Overall: {'SUCCESS' if success else 'NEEDS IMPROVEMENT'}")

if success:
    print("\n✅ Faz-1 pretraining çalışıyor, loss düşüyor!")
else:
    print("\n⚠️ Model optimization needed:")
    if np.isnan(final_loss):
        print("  - Gradient clip ekle: torch.nn.utils.clip_grad_norm_")
    if final_loss >= 1.0:
        print("  - Learning rate azalt veya batch size artır")
        print("  - Window size=512 sabit tut")